In [1]:
# Run this 1
from file_process import * #==> issues on the import of splitDocs, don't read efficiently the content
## Fonction de traintement du texte
from text_process import *
from weiting_function import *
from runs_function import *

In [2]:
# 2
list_file = list_file_data("../Practice_05_data/XML-Coll-withSem/")
dataset = "../Practice_05_data/XML-Coll-withSem/"
# Cell to initialise global variable
team_name = "DjibrilMohamedOmaimaDouae"
assets = "assets/"
k = 1.2
b = 0.75
use_stem = False
use_stopword = True
run_id=0
list_termes = {}

# get the query file
directory = "../Practice_04_data/"
#list_data_q = practice2.list_file_data(directory)
filename_q = str(directory+"topics_M2DSC_7Q.txt")
query_list =preprocesFile(filename_q)
query = query_list.split("\n")

In [9]:

#fileName = '10237.xml'
#filePath = dataset+fileName

In [10]:
#content = preprocesFile(dataset+fileName)

In [11]:
#don't run the cell
leaf_1 = 'header>title'
leaf_2 = 'bdy>sec>ss1>p'
leaf_2_1 = 'bdy>p'
leaf_2_2 = 'bdy>sec'


In [12]:

#leaf_2_dict = {}
#bdy = BeautifulSoup(content,'xml').find_all('bdy')
#p = bdy[0].select('bdy>p')
#sec = bdy[0].select('bdy>sec')

#r = str(bdy[0]).replace(str(p[0]),"")


#len(sec)

In [3]:
# 3
def parse_xml(content,list_id):
    corpus = {} # Init corpus
    corpus['title'] = BeautifulSoup(content,'xml').find_all('title')
    bdy = BeautifulSoup(content,'xml').find_all('bdy')
    corpus['bdy'] = {} # Init body part
    id = str(BeautifulSoup(content,'xml').find_all('id')[0]).strip('</id>')
    # Recuperation des paragraphes dans le body
    p = bdy[0].select('bdy>p')
    b = bdy[0]
    #Recuperation des sections dans le body
    sec = bdy[0].select('bdy>sec')
    paragraphe = {}
    if(len(p)>0):
        for i in range(len(p)):
            bdy[0] = str(bdy[0]).replace(str(p[i]),"") # Supprimer les paragraphes 
            paragraphe[str('p.{}').format(i+1)] = p[i] # Ajoute le i_eme paragraphe de body

        corpus['bdy'].update(paragraphe) # Mise a jours de body
            
    if(len(sec)>0):
        section = {}
        sous_section ={} 
        sous_section_p ={} 
        rest = {}
        for i in range(len(sec)):
            section['sec.{}'.format(i+1)] = {}
            sous_section.clear()
            
            rest.clear()
            paragraphe.clear()
            #print('sec_{}'.format(i))
            r_p = sec[i].get_text()
            bdy[0] = str(bdy[0]).replace(str(sec[i]),"") # On retire tout le contenue de sec[i] de body
            
            #print('ss : {}'.format(len(sous_sections)))
            # recuperation des paragraphe de section
            par = BeautifulSoup(str(sec[i]),'xml').select('sec>p')
            if(len(par)>0):
                #print('par : {}'.format(len(par)))
                # a = r_p
                j = 0
                for p in par:
                    
                    r_p = str(r_p).replace(p.get_text(),"") # Recupere le texte et le supprime dans la section
                    
                    paragraphe[str('p.{}').format(j+1)] = p
                    j+=1
                section['sec.{}'.format(i+1)] = paragraphe.copy()
                 
            # recuperation des sous sections de section
            sous_sections = BeautifulSoup(str(sec[i]),'xml').select('sec>ss1')
            if(len(sous_sections)>0):
                # recuperation les paragraphes des sous sections
                for j in range(len(sous_sections)):
                    sous_section_p.clear() 
                    sous_section['ss1.{}'.format(j+1)] = {}
                    #print('\t\tss1')
                    #print(str(sous_sections[j]))
                    r_p = r_p.replace(sous_sections[j].get_text(),"")
                    ss_p = BeautifulSoup(str(sous_sections[j]),'xml').select('ss1>p')
                    if(len(ss_p)>0):
                        #print('ss_p : {}'.format(len(ss_p)))
                        for k in range(len(ss_p)):
                            sous_section_p['p.{}'.format(k+1)] = ss_p[k]
                    sous_section['ss1.{}'.format(j+1)].update(sous_section_p.copy())
                #print(sous_section.keys())
            if(len(sous_section.keys())>0):
                section['sec.{}'.format(i+1)].update(sous_section.copy())

            if(len(r_p)>0):
                rest['NaN'] = r_p
                section['sec.{}'.format(i+1)].update(rest)
        #print("insertiong data on corpus")
        corpus['bdy'].update(section)
        if(len(bdy[0])>0):
            
            #print("rest",b)
            rest['NaN'] = bdy[0]
            corpus['bdy'].update(rest)
    list_termes[id] = corpus
    return list_termes
           

In [4]:
# 4
def build_pl(text, node,posting_list_el,use_stem, use_stopword):
    text_clean = clean(text, use_stem, use_stopword)
    lt = text_clean.split()
    #lt_['{}_{}'.format(node,j)] = lt
    current_dico = countWord(lt)
    posting_list_el = countWordIntoDocs(current_dico,'{}'.format(node), posting_list_el)
    return posting_list_el

def xml_text_minings_v2(list_terms = dict(), use_stem=bool(), use_stopword=bool()):
    start = time.time()
    new_list_terms = {}
    posting_list = {}
    posting_list_el = {}
    lt = dict()
    lt_final = dict()
    dl = list()
    i = 0

    for docno, title_bdy in list_terms.items():
        i+=1
        j = 0
        lt.clear()
        posting_list_el.clear()
      
        title = title_bdy['title'][0].get_text()
        title  =clean(title, use_stem, use_stopword)
        lt['title'] = title.split() # Recuperation de content title
        posting_list_el = build_pl(title, "title",posting_list_el,use_stem, use_stopword)
        
        for node_1, value_1 in title_bdy["bdy"].items(): 
            if "sec" in node_1:
                for sec, value_2 in value_1.items():# Recuperation des sections de body
                    if "ss" in sec:
                        for p,val in value_2.items(): # Recuperation des paragraphe de chaque sous section
                            v = clean(val.text, use_stem, use_stopword)
                            lt[str("bdy_{}_{}_{}".format(node_1,sec,p))] = v.split()
                            posting_list_el = build_pl(v, str("bdy_{}_{}_{}".format(node_1,sec,p)),posting_list_el,use_stem, use_stopword)          
                    else:
                        if "NaN" in sec: # Recuperation des NaN de la section
                            val = value_2.strip()  
                        else : 
                            val = clean(value_2.text, use_stem, use_stopword) # Recuperation des pa
                        lt[str("bdy_{}_{}".format(node_1,sec))] = val.split()
                        posting_list_el = build_pl(val, str("bdy_{}_{}".format(node_1,sec)),posting_list_el,use_stem, use_stopword)
 
            else:
                if "NaN" in node_1: # Recuperation des NaN de body
                    val = value_2.strip()  
                else : 
                    val = clean(value_1.text, use_stem, use_stopword) # Recuperation des paragraphes de body
                lt[str("bdy_{}".format(node_1))] = val.split()
                posting_list_el = build_pl(val, str("bdy_{}".format(node_1)),posting_list_el,use_stem, use_stopword)

      
        lt_final[docno] = lt.copy()
        posting_list[docno] = posting_list_el.copy()

    return posting_list,lt_final,(start)



In [5]:
# this is important
# For full content
start = time.time()
i = 0
for file_name in list_file:
    i+=1
    print(i, file_name)
    if i==200:
        break
    list_termes = parse_xml(preprocesFile(dataset+file_name),list_termes)
print((time.time()-start)/60)

1 10003934.xml
2 10008302.xml
3 10013.xml
4 10013985.xml
5 10018.xml
6 10019528.xml
7 10022607.xml
8 1002307.xml
9 10025826.xml
10 10025863.xml
11 10037185.xml
12 10045226.xml
13 10046553.xml
14 10046651.xml
15 1004679.xml
16 1004743.xml
17 10048713.xml
18 10050395.xml
19 10052718.xml
20 100558.xml
21 10056712.xml
22 10057294.xml
23 1005923.xml
24 10061279.xml
25 10061978.xml
26 1006214.xml
27 10063963.xml
28 10069526.xml
29 10072671.xml
30 10073799.xml
31 10074317.xml
32 10077874.xml
33 10078385.xml
34 1007928.xml
35 1008031.xml
36 10083025.xml
37 10085727.xml
38 1008632.xml
39 1008941.xml
40 10089909.xml
41 10092617.xml
42 1009996.xml
43 1010280.xml
44 10105635.xml
45 10105982.xml
46 1011379.xml
47 10118518.xml
48 10119206.xml
49 10123713.xml
50 1012806.xml
51 1013089.xml
52 10131591.xml
53 10132085.xml
54 10133976.xml
55 10135033.xml
56 10136.xml
57 10140499.xml
58 10141125.xml
59 10143509.xml
60 10143862.xml
61 1014399.xml
62 10145406.xml
63 10148645.xml
64 10153951.xml
65 1015600.

In [6]:
# This function is for only full content indexing
def get_xml_text(value):
    j = 0
    lt_ = dict()
    lt_final = dict()
    posting_list = {}
    posting_list_el = {}
    for node, content in value.items():
        posting_list_el.clear()
        lt_.clear()
        for text in content:
            j+=1
            soup = BeautifulSoup(str(text), "xml")
            text_content = soup.get_text()
            text_clean = clean(text_content, use_stem, use_stopword)
            lt = text_clean.split()
            lt_['{}_{}'.format(node,j)] = lt
            current_dico = countWord(lt)
            posting_list_el = countWordIntoDocs(current_dico,'{}_{}'.format(node,j), posting_list_el)
        lt_final[node] = lt_.copy()
        posting_list[node] = posting_list_el.copy();
    return lt_final,posting_list

def xml_text_minings_full(list_terms = dict(), use_stem=bool(), use_stopword=bool()):
    start = time.time()
    new_list_terms = {}
    posting_list = {}
    dl = list()
    i = 0
    for docno, content in list_terms.items():
        i+=1
        text_clean = clean(content, use_stem, use_stopword)
        new_list_terms[docno] = text_clean.split()
        lt = text_clean.split()
        current_dico = countWord(lt)
        #sec_1 = "[word : fre]"
        posting_list = countWordIntoDocs(current_dico,docno, posting_list)
    
    return posting_list, new_list_terms,(time.time() - start)

In [ ]:
#dd = list_termes["10843"]
#dd['bdy']['sec.1']

In [7]:
# Run this to take only the sections
def xml_text_minings(list_terms = dict(), use_stem=bool(), use_stopword=bool()):
    start = time.time()
    new_list_terms = {}
    posting_list = {}
    posting_list_el = {}
    lt_ = dict()
    lt_final = dict()
    dl = list()
    i = 0

    for docno, content in list_terms.items():
        i+=1
        j = 0
        #print(i,"==>",id, "len sec==>", len(content))
        # if content[0] = 1 else normal indexion
        posting_list_el.clear()
        lt_.clear()
        for text in content:
            j+=1
            soup = BeautifulSoup(str(text), "xml")
            text_content = soup.get_text()
            #print(text_content)
            text_clean = clean(text_content, use_stem, use_stopword)
            #new_list_terms[id] = text_clean.split()
            lt = text_clean.split()
            lt_[j] = lt
            current_dico = countWord(lt)
            #sec_1 = "[word : fre]"
            posting_list_el = countWordIntoDocs(current_dico,"sec_{}".format(j), posting_list_el)
            #posting_list.setdefault(docno,[]).append(countWordIntoDocs(current_dico,"sec_{}".format(j), posting_list_el))
            #posting_list_el = countWordIntoDocs(current_dico,"sec_{}".format(j), posting_list_el) 
            #print("Sec[{}]==>{}".format(j,text_content))
            #print("**********-/*-*/-*-************************************************-/-*/-/-*/-*/-",j)
            
        lt_final[docno] = lt_.copy()
        posting_list[docno] = posting_list_el.copy();
     
        #break
        #if(i==80): break
    return posting_list,lt_final,(time.time() - start)

def splitDocs3(fileDoc, list_terme):
    content = []
    sec_dict = {} # keys = {chiffre incrementale} # values = {contenues des paragraphes}
      # Read the XML file
    with open(fileDoc, "r",encoding='utf-8') as file:
      # Read each line in the file, readlines() returns a list of lines
      content = file.readlines()
      # Combine the lines in the list into a string
      content = " ".join(content)
      soup = BeautifulSoup(content, "xml")
      text =  soup.get_text()
      id = str(soup.find_all('id')[0]).strip('</id>')
      sec = soup.find_all('sec')
      # list_terme[id] = ['P'|'S'] # 0==> il n'ya pas de section et le 1 il y a des sections
   
      list_terme[id]=sec  # Pour chaque article on recuper les sections
    return list_terme


In [8]:
# 5
# Part of statistique

def doc_len_xml(list_terms):
    dl = []
    dl_sec = {}
    n_el = 0
    i =0
    title = 0
    bdy_n = 0
    bdy_p = 0
    sec_n = 0
    sec_p  =0
    ss_p = 0
    for doc_n, sec in list_terms.items():
        
        n = ''
        i+=1
        #if i==10: break
            
        for nodes, len_sec in sec.items():
            
            for node in nodes.split("_"):
                #print(len_sec)
                if "bdy" in node: 
                    n = "bdy"
                elif "title" in node: 
                    title+=1
                    
                elif("sec" in node):
                    n = "sec"
                  
                elif("ss" in node):
                    n = "ss"
                    
                if("p" in node):
                    if n=="bdy":
                        bdy_p+=1
                    elif n=="sec":
                        sec_p+=1
                    else:
                        ss_p+=1
                        
                if("NaN" in node):
                    if n=="bdy":
                        bdy_n+=1
                    elif n=="sec":
                        sec_n +=1
    dl = [title,bdy_n,bdy_p,sec_n,sec_p,ss_p]
    n_el = sum(dl)
    # dl = [(doc_n, len(len_doc)) for doc_n,len_doc in list_terms.items()]
    return len(list_termes.keys()), dl,(n_el+len(list_termes.keys()))

#{docn : {sec_1:5}}
def get_xml_stat(posting_list, list_terms):
    stat = {}
    # document length
    # [title,bdy_n,bdy_p,sec_n,sec_p,ss_p]
    stat['n_article'],stat['n_node'],stat['n_element'] = doc_len_xml(list_terms)
    # Number of a doc
    
    # term length
    stat['tl'] = term_len(posting_list)
    # vocabulary size
    stat['voc_size'] = vocabulary_size(posting_list)
    # collection frequency of terms
    stat['colec_freq'] = collection_term_freq_xml(posting_list)
    return stat

def collection_term_freq_xml(posting_list):
    c_size = {}
    dl = {}
    for term, article in posting_list.items():  # get the term
        somme = 0
        j = 0
        title = 0
        bdy_n = 0
        bdy_p = 0
        sec_n = 0
        sec_p  =0
        ss_p = 0
        n = ''
        for secs in article:
            for sec,val in secs.items():
                for nodes in val:
                    for node in nodes[0].split("_"):
                        
                        if "bdy" in node: 
                            n = "bdy"

                        elif "title" in node: 
                            title+=nodes[1]

                        elif("sec" in node):
                            n = "sec"

                        elif("ss" in node):
                            n = "ss"

                        if("p" in node):
                            if n=="bdy":
                                bdy_p+=nodes[1]
                            elif n=="sec":
                                sec_p+=nodes[1]
                            else:
                                ss_p+=nodes[1]

                        if("NaN" in node):
                            if n=="bdy":
                                bdy_n+=nodes[1]
                            elif n=="sec":
                                sec_n +=nodes[1]
        j = sum([title,bdy_n,bdy_p,sec_n,sec_p,ss_p])
        dl[term] = [title,bdy_n,bdy_p,sec_n,sec_p,ss_p ,j]
                                                   # somme = somme de frequence dans toutes les sections
    return dl                                              
                                                            # j = nombre total d'element


In [ ]:
# For content with sec
"""
for file_name in list_file:
    list_termes = splitDocs3(dataset+file_name,list_termes)
pl,lt,ti = xml_text_minings(list_termes,use_stem, use_stopword)

new_pl = {}
for dic, values in pl.items():
        for word,v in values.items():
            new_pl.setdefault(word,[]).append({dic:v})

print(ti/60)

"""


In [9]:
# 6
pl,lt,ti = xml_text_minings_v2(list_termes,use_stem, use_stopword)

new_pl = {}
for dic, values in pl.items():
        for word,v in values.items():
            new_pl.setdefault(word,[]).append({dic:v})
print("....Indexing time is : {}".format((time.time() - ti)/60))
stat = get_xml_stat(new_pl,lt)
print("Execution time for indexing and statistique : {}".format((time.time() - ti)/60))#collection_term_freq_xml(new_pl)

....Indexing time is : 0.056399397055308026
Execution time for indexing and statistique


In [19]:
# 'bdy_sec.8_NaN' ==> bdy sec.8 NaN
# 'bdy_NaN' ==> bdy NaN
stat['n_node']

[199, 188, 356, 894, 1941, 742]

In [11]:
# 7
def smart_ltn_xml(posting_list,stat):
    i = 0
    n = stat['n_element']
    df_t = stat['colec_freq']
    df = 0
    #(1+LOG(tf))*LOG(n/df)
    #tf is the frequence of term on a document
    #df number of document where the term appear
    ltn_value_sec = {} # key sec, value (terme, ltn)
    ltn_value = {} # key doc number, value (terme, ltn)
    
    for term, articles in posting_list.items():
        i+=1
        dl = df_t[term]
        df = dl[-1] # nombre de doc dans lequel le terme apparait sur toute la collection
        # print(value) [{'10013': [('sec_1', 1), ('sec_12', 1)]},...]
        
        for sec in articles:
            
            for doc_no,val in sec.items():
                ltn_value_sec.clear()
                for v in val:
                    tf = v[1]
                    sec =v[0]
                    ltn = (1+math.log10(tf))*(math.log10(n/df))
                    
                    ltn_value_sec.setdefault(sec,[]).append((term,round(ltn,4))) # ltn en fonction de la section
                ltn_value.setdefault(doc_no,[]).append(ltn_value_sec.copy()) # ltn en fonction des articles
    return ltn_value

def wf_refractor(wf):
    new_wf = {}
    wf_refractor = {}
    for doc, article in wf.items():
        new_wf.clear()
        for l in article: 
            for se, k in l.items():
                for i in k:
                    new_wf.setdefault(se,[]).append(i)
        wf_refractor[doc] = new_wf.copy()
    return wf_refractor

def xml_w_score(wf=dict()):
    score_wt = {}
    for doc_no, article in wf.items():
        scores = rsv_score(query,article)
        if(len(scores)>0):
            for sec in scores:
                score_wt.setdefault(doc_no,[]).append(sec)
    return score_wt

def ltc_xml(ltn):
    score_ltn = {}
    for doc_no, sec_ltn_value in ltn.items():
        for value in sec_ltn_value:
            res = smart_ltc(sec_ltn_value)
            score_ltn[doc_no] = res
    return score_ltn

In [20]:
#stat['colec_freq']
#ltn = wf_ref(smart_ltn_xml(new_pl,stat))


In [42]:
# 8
def tf_bloc(nodes,k,b,term_tf,avdl_el):
    tf = term_tf
    doc_len = nodes
    bloc_1 = tf*(k+1)
    bloc_2 = doc_len/avdl_el
    bloc_3 = k * ((1-b) + b * bloc_2)
    bloc_4 = bloc_3+tf
    return bloc_1/bloc_4

def tf_part_xml(posting_list,dl,n,k,b, cas):
    #(tf*(k+1))/(k*((1-b)+b*(dl/avdl))+tf)
    #[  1     ]             [   2   ]
    #              [      3         ]
    #            [         4            ]
    
    # [title,bdy_n,bdy_p,sec_n,sec_p,ss_p ,j]
    dl_ = sum([dl[x][-1] for x in dl.keys()]) # Nombre de words dans tous les documents
    n_elements = sum([dl[x][-1] for x in dl.keys()]) # Nombre de words dans tous les éléments
    n_title = sum([dl[x][0] for x in dl.keys()]) # Nombre de words dans tous les title
    n_bdy_n = sum([dl[x][1] for x in dl.keys()]) # Nombre de words dans tous les body
    n_bdy_p = sum([dl[x][2] for x in dl.keys()]) # Nombre de words dans tous les paragraphes des body
    n_sec_n = sum([dl[x][3] for x in dl.keys()]) # Nombre de words dans tous les sections
    n_sec_p = sum([dl[x][4] for x in dl.keys()]) # Nombre de words dans tous les paragraphes dans section
    n_ss_p = sum([dl[x][5] for x in dl.keys()]) # Nombre de words dans tous les paragraphes dans sous section
        #dl[term] = [title,bdy_n,bdy_p,sec_n,sec_p,ss_p ,j]
    n_node = stat['n_node'] # nombre total des balises of each element [0,1,2,3,4,5]
    doc_len = 0
    avdl_el = n_elements/n
    #print(avdl)
    #avdl = 20
    tf_part_val = {}
    tf_part_val_l = {}
    tf_part_val_node = {}
    kl = 0
    for term, articles in posting_list.items():
        kl+=1
        for sec in articles:
            for doc_no,val in sec.items():
                tf_part_val_node.clear()
                for v in val:
                    tf = v[1]
                    sec =v[0]
                    part_el = sec.split('_')
                    if part_el[-1] == 'title':
                        avdl = n_title/n_node[0]
                    if part_el[-1] == 'NaN':
                        if 'bdy' in part_el[-2]:
                            avdl = n_bdy_n/n_node[1]
                        if 'sec' in part_el[-2]:
                            avdl = n_sec_n/n_node[3]
                        
                    if 'p' in part_el[-1]:
                        if 'bdy' in part_el[-2]:
                            avdl = n_bdy_p/n_node[2]
                        if 'sec' in part_el[-2]:
                            avdl = n_sec_n/n_node[4]
                            
                        if 'ss' in part_el[-2]:
                            avdl = n_sec_n/n_node[5]
                    if cas == 1:
                        avdl = avdl_el
                    # print(sec.split('_')[-1])
                    bm25 =  tf_bloc(n_sec_n,k,b,tf,avdl)
                    tf_part_val_node.setdefault(sec,[]).append((term,round(bm25,4))) # ltn en fonction de la section
                tf_part_val.setdefault(doc_no,[]).append(tf_part_val_node.copy()) # ltn en fonction des articles
    return tf_part_val

def idf_part_xml(posting_list,df_,n):
    #log((n-df+0.5)/(df+0.5))
    #    [   1    ] [  2   ]
    idf_part_val ={}
    idf_part_val_node ={}
    for term, articles in posting_list.items():
        for sec in articles:
            for doc_no,val in sec.items():
                idf_part_val_node.clear()
                for v in val:
                    sec =v[0]
                    tf =  df_[term]
                    res =  idf_block(tf,n)
                    idf_part_val_node.setdefault(sec,[]).append((term,res))
                idf_part_val.setdefault(doc_no,[]).append(idf_part_val_node.copy()) 
    return idf_part_val

def idf_block(tf,n):
    df = tf
    df = df[-1]
    bloc_1 = abs(n-df+0.5)
    bloc_2 = df + 0.5
    return math.log10(bloc_1/bloc_2)

def bm25_xml(posting_list, stat,k,b):
    tf_part_ = wf_refractor(tf_part_xml(posting_list,stat['colec_freq'],stat['n_element'],k,b,0))
    idf_part_ = wf_refractor(idf_part_xml(posting_list,stat['colec_freq'], stat['n_element']))
    bm25_val = {}
    # doc 1 : ('a' , tf_part_[a]*idf[a])
    for doc, tf_value in tf_part_.items():
        for tf in tf_value:
            res = [(tf[0],tf[1] * idf[1]) for tf, idf in zip(tf_part_[doc][tf], idf_part_[doc][tf]) if tf[0]==idf[0]]
            bm25_val.setdefault(doc, []).append({tf:res})
    return bm25_val

# Part of weigting function
def weinting_function_xml(pl, stat,k,b):
    ltn = wf_refractor(smart_ltn_xml(pl,stat))
    ltc = ltc_xml(ltn)
    bm25_val = wf_refractor(bm25_xml(pl,stat,k,b))
    return ltn,ltc,bm25_val

    

In [41]:
#tf_part_xml(new_pl,stat['colec_freq'],stat['n_element'],k,b,0)

In [ ]:
# 9
def score_xml(query_list,rsv_wf):
    team_name = "DjibrilMohamedOmaimaDouae"
    score = []
    for query in query_list:
        query = query.split(' ', 1)
        result = result_querys(query[0],rsv_score_(query[1], rsv_wf), team_name)
        score.append(result)
    return score

def result_querys(num_query,rsv_result,team_name):
    start = time.time()
    #score = reverse_score(rsv_result)
    rsv_r = []
    k = 0
    x = rsv_result
    if len(x)>1500:
        x = x[:1500]
    
    # bdy_sec.13_p.2 new.join(s.rsplit(old, maxreplace))
    for i in range(len(x)):
        path = "/article[1]/"
        k=0
        for el in x[i][1].split("_"):
            k+=1
            if "bdy" in el:
                path+= "bdy[1]/"
            elif "title" in el:
                path+= "title[1]/"
            elif "NaN" in el:
                path = path
            else:
                ele = el.split('.')
                path +=str("{}[{}]/".format(ele[0],ele[1]))
        path =''.join(path.rsplit('/',1))
        rsv_r.append(str('{} Q0 {} {} {} {} {}'.format(num_query,
                                                           x[i][2],i+1,
                                                           round(x[i][0],5),
                                                           team_name,path))) 
             
    return rsv_r


def rsv_score_(query, wf):
    rsv_doc = {}
    rsv_sec= {}
    rsv = []
    for term in query.split():
        for doc, sec in wf.items():
            rsv_sec.clear()
            for k,val in sec.items():
                for v in val:
                    if term == v[0]:
                        if k in rsv_sec.keys():
                            v_temp = rsv_sec[k] + v[1]
                            v[1] = v_temp
                        rsv.append((v[1],k,doc))
                    else:
                        continue 
            if(len(rsv_sec.keys())>0):
                rsv_doc[doc] = rsv_sec.copy()
    rsv = sorted(rsv, key=lambda item: item[0],reverse=True)
    return rsv

# For all weigting function
def all_score_wf(query_list, ltn,ltc,bm25):
    start = time.time()
    score_ltn= score_xml(query,ltn)
    print("Ltn time : ",time.time() -start)
    start = time.time()
    score_ltc = score_xml(query,ltc)
    print("Ltc time : ",time.time() -start)
    start = time.time()
    score_bm25 = score_xml(query,bm25)
    print("Bm25 time : ",time.time() -start)
    
    return score_ltn,score_ltc,score_bm25

In [ ]:
# 10
ltn,ltc,bm25 =weinting_function_xml(new_pl, stat, k, b)
score_ltn,score_ltc,score_bm25 = all_score_wf(query, ltn,ltc,bm25)

In [ ]:
# Take one score here and give it to his 2nd paraeter
#bm25_s = scores(query,bm25)    |here|
run_id = build_run_file(run_id, score_ltn, 0, use_stem, use_stopword, k, b)

In [ ]:
def bm25_tuning(run_id,score_bm25,use_stem,use_stopword):
    start = time.time()
    k = 1.2
    b = 0.0
    for i in range(11):
        run_id = build_run_file(run_id, score_bm25,2,use_stem,use_stopword,k,b)
        b+=0.1
    k = 0.0
    b = 0.75
    for i in range(21):
        run_id = build_run_file(run_id, score_bm25,2,use_stem,use_stopword,k,b)
        k+=0.2